## 1.加载keras模块

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from keras.preprocessing.image import ImageDataGenerator
#from keras.models import Sequential
#from keras.layers import Convolution2D, MaxPooling2D,ZeroPadding2D
#from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from keras import backend as K
import numpy as np

### 加载预训练权值



### 定义新加入的layers

In [8]:
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
# 定义网络输入层shape
input_tensor = Input(shape=input_shape)
#声明VGG网络，并指定加载imagenet预训练模型
model = VGG16(weights='imagenet', include_top=False ,input_shape=input_shape)
#向VGG网络追加一些全连接层和输出层
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))


#指定输出层为2个神经元做2分类，激活函数选用softmax
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='softmax'))
#把上述网络每一层layer拷贝至一个新的网络，形成一个整体网络
new_model = Sequential()
for l in model.layers:
    new_model.add(l)
new_model.add(top_model)

Exception: URL fetch failure on https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [Errno 11001] getaddrinfo failed

In [4]:
seed = 7
np.random.seed(seed)


def Conv2d_BN(x, nb_filter, kernel_size, padding='same', strides=(1, 1), name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv2D(nb_filter, kernel_size, padding=padding, strides=strides, activation='relu', name=conv_name)(x)
    x = BatchNormalization(axis=3, name=bn_name)(x)
    return x


def Inception(x, nb_filter):
    branch1x1 = Conv2d_BN(x, nb_filter, (1, 1), padding='same', strides=(1, 1), name=None)

    branch3x3 = Conv2d_BN(x, nb_filter, (1, 1), padding='same', strides=(1, 1), name=None)
    branch3x3 = Conv2d_BN(branch3x3, nb_filter, (3, 3), padding='same', strides=(1, 1), name=None)

    branch5x5 = Conv2d_BN(x, nb_filter, (1, 1), padding='same', strides=(1, 1), name=None)
    branch5x5 = Conv2d_BN(branch5x5, nb_filter, (1, 1), padding='same', strides=(1, 1), name=None)

    branchpool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branchpool = Conv2d_BN(branchpool, nb_filter, (1, 1), padding='same', strides=(1, 1), name=None)

    x = concatenate([branch1x1, branch3x3, branch5x5, branchpool], axis=3)

    return x


inpt = Input(shape=(224, 224, 3))
# padding = 'same'，填充为(步长-1）/2,还可以用ZeroPadding2D((3,3))
x = Conv2d_BN(inpt, 64, (7, 7), strides=(2, 2), padding='same')
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Conv2d_BN(x, 192, (3, 3), strides=(1, 1), padding='same')
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Inception(x, 64)  # 256
x = Inception(x, 120)  # 480
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Inception(x, 128)  # 512
x = Inception(x, 128)
x = Inception(x, 128)
x = Inception(x, 132)  # 528
x = Inception(x, 208)  # 832
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
x = Inception(x, 208)
x = Inception(x, 256)  # 1024
x = AveragePooling2D(pool_size=(7, 7), strides=(7, 7), padding='same')(x)
x = Dropout(0.4)(x)
x = Dense(1000, activation='relu')(x)
x = Dense(1000, activation='softmax')(x)
model = Model(inpt, x, name='inception')
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

Model: "inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           batch_normalization[0][0]        
__________________________________________________________________________________________

### 设置不需微调的layers的trainable属性
并调用compile函数重新编译网络

In [10]:
from keras.optimizers import SGD
#在新的整体网络上设置对所有原VGG层均不参与微调
for layer in new_model.layers[:4]:
    layer.trainable = False

#compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
#注意此处应选用何种loss function
new_model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

### 定义ImageDataGenerator


In [11]:
train_data_dir = r'D:/AI/11.6/dogs-vs-cats\train'
validation_data_dir = r'D:/AI/11.6/dogs-vs-cats\validation'
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 20


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)
#此处调用flow_from_directory函数生成train_generator和validation_generator,注意设置class_mode
train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=(img_width,img_height),
                                                   batch_size=batch_size,class_mode='')
validation_generator= test_datagen.flow_from_directory(test_data_dir,target_size=(img_width,img_height),
                                                       batch_size=batch_size,class_mode='')


Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [12]:
new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/1
541/541 [==============================] - 253s 467ms/step - loss: 0.3200 - accuracy: 0.8505 - val_loss: 0.1355 - val_accuracy: 0.9337


### 使用训练后模型预测图像





In [16]:
import cv2
img = cv2.resize(cv2.imread(r'C:\Users\coffe\Desktop\dogs-vs-cats\test\1.jpg'), (img_width, img_height)).astype(np.float32)
# img[:,:,0] -= 103.939
# img[:,:,1] -= 116.779
# img[:,:,2] -= 123.68
#img = img.transpose((2,0,1))
x = img_to_array(img)

x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

score = new_model.predict(x)


print(score)

[[0. 1.]]


###此处尝试只锁定VGG头四层

In [ ]:
#在新的整体网络上设置原VGG前四层不参与微调


#编译网络后进行训练



In [ ]:
#使用训练后模型预测图像

